In [2]:
import pandas as pd

In [3]:
mjgross = pd.read_csv('D:/Flatiron/Project/Zips/bom.movie_gross.csv/bom.movie_gross.csv')
imdbname = pd.read_csv('D:/Flatiron/Project/Zips/imdb.name.basics.csv/name.basics.csv')
imdbtitle = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.akas.csv/title.akas.csv')
imdbtitle2 = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.basics.csv/title.basics.csv')
imdbcrew = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.crew.csv/title.crew.csv')
imdbprinc = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.principals.csv/title.principals.csv')
imdbrate = pd.read_csv('D:/Flatiron/Project/Zips/imdb.title.ratings.csv/title.ratings.csv')
rtinfo = pd.read_csv('D:/Flatiron/Project/Zips/rt.movie_info.tsv/rt.movie_info.tsv', delimiter='\t')
rtreviews = pd.read_csv('D:/Flatiron/Project/Zips/rt.reviews.tsv/rt.reviews.tsv', sep=None)
tmdbmovies = pd.read_csv('D:/Flatiron/Project/Zips/tmdb.movies.csv/tmdb.movies.csv')
tnbudgets = pd.read_csv('D:/Flatiron/Project/Zips/tn.movie_budgets.csv/tn.movie_budgets.csv')

In [4]:
# joined imdbcrew and imdbtitle2
imdbcrewtitle2 = imdbtitle2.join(imdbcrew.set_index('tconst'), on='tconst')


In [5]:
#Editing, dropping mostly empty or useless columns
imdbname.drop(['birth_year','death_year'], axis=1, inplace=True)
imdbtitle.drop(['attributes','language'], axis=1, inplace=True)
imdbprinc.drop(['job','characters'], axis=1, inplace=True)
rtinfo.drop(['synopsis','theater_date','dvd_date','studio','currency'], axis=1, inplace=True)

In [6]:
#Just to have a handy reference point
print('\n mjgross \n'); print(len(mjgross)); print(mjgross.isna().sum())
print('\n imdbname \n'); print(len(imdbname)); print(imdbname.isna().sum())
print('\n imdbtitle \n'); print(len(imdbtitle)); print(imdbtitle.isna().sum())
print('\n imdbcrewtitle2 \n'); print(len(imdbcrewtitle2)); print(imdbcrewtitle2.isna().sum())
print('\n imdbprinc \n'); print(len(imdbprinc)); print(imdbprinc.isna().sum())
print('\n imdbrate \n'); print(len(imdbrate)); print(imdbrate.isna().sum())
print('\n rtinfo \n'); print(len(rtinfo)); print(rtinfo.isna().sum())
print('\n rtreviews \n'); print(len(rtreviews)); print(rtreviews.isna().sum())
print('\n tmdbmovies \n'); print(len(tmdbmovies)); print(tmdbmovies.isna().sum())
print('\n tnbudgets \n'); print(len(tnbudgets)); print(tnbudgets.isna().sum())


 mjgross 

3387
title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

 imdbname 

606648
nconst                    0
primary_name              0
primary_profession    51340
known_for_titles      30204
dtype: int64

 imdbtitle 

331703
title_id                  0
ordering                  0
title                     0
region                53293
types                163256
is_original_title        25
dtype: int64

 imdbcrewtitle2 

146144
tconst                 0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
directors           5727
writers            35883
dtype: int64

 imdbprinc 

1028186
tconst      0
ordering    0
nconst      0
category    0
dtype: int64

 imdbrate 

73856
tconst           0
averagerating    0
numvotes         0
dtype: int64

 rtinfo 

1560
id               0
rating           3
genre            8
director       1

In [22]:
studiogross = mjgross.set_index('studio')

mjgross.groupby('studio').domestic_gross.mean().sort_values(ascending=False).head(10)

studio
BV         1.737644e+08
P/DW       1.682900e+08
WB (NL)    8.879333e+07
Uni.       8.777138e+07
WB         8.691461e+07
Fox        8.051103e+07
Sony       7.761177e+07
Par.       7.609773e+07
MGM        6.666667e+07
Sum.       6.212473e+07
Name: domestic_gross, dtype: float64

In [ ]:
#in progress DON'T RUN
def splitmult(df, column):
    for i, item in enumerate(df[column]):
        each = item.split(sep=',')
        item = each[0]
        for e in range(1, len(each)):
            new_rows = {}
            new_rows.append(str(df.iloc[i].loc[column]) + ' : ' + )

In [ ]:
imdbname['primary_name'][1]

In [ ]:
imdbtitle

In [ ]:
imdbprincnametitle

In [ ]:
imdbcrewtitle2

In [51]:
imdbprincname = imdbprinc.merge(imdbname, on='nconst')

In [ ]:
imdbprincnametitleouter = imdbprincname.merge(imdbtitle, left_on='tconst', right_on='title_id',how='outer')

In [ ]:
imdball = imdbprincnametitleouter.merge(imdbcrewtitle2, on='tconst')
imdball

In [ ]:
imdball.category.unique()

In [ ]:
# idea: make dictionary, keys are nm#####s, values are names. use replace function
#dictionary to replace names
names = []
for n in imdbname['nconst']:
    names.append(n)
len(names)
real_names = []
for n in imdbname['primary_name']:
    real_names.append(n)
len(real_names)
name_dict = dict(zip(names, real_names))

In [ ]:
# def replcnm():
#     for i, item in enumerate(imdbprinc.loc[:,'nconst']):
#         if str(item) in names:
#             imdbprinc.replace(str(item), name_dict[str(item)])
        # try:
        #     imdbprinc.iloc[i].loc['nconst'].replace(item, name_dict[item])
        # except:
        #     continue

    #     if item not in names:
    #         continue
    #     if item in names:
    #         imdbprinc.replace(item, name_dict[item])
# newplc = imdbprinc.replace({'nconst': name_dict})
# newplc

In [ ]:
name_dict['nm0476864']

In [ ]:
len(name_dict)